In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [3]:
g=dataset[0]
print("Node features")
print(g.ndata)
print("Edge features")
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
Edge features
{}


In [4]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [5]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.092 (best 0.092), test acc: 0.079 (best 0.079)
In epoch 5, loss: 1.902, val acc: 0.310 (best 0.456), test acc: 0.329 (best 0.422)
In epoch 10, loss: 1.826, val acc: 0.582 (best 0.582), test acc: 0.588 (best 0.588)
In epoch 15, loss: 1.724, val acc: 0.688 (best 0.688), test acc: 0.695 (best 0.686)
In epoch 20, loss: 1.596, val acc: 0.698 (best 0.698), test acc: 0.696 (best 0.696)
In epoch 25, loss: 1.446, val acc: 0.712 (best 0.712), test acc: 0.711 (best 0.711)
In epoch 30, loss: 1.279, val acc: 0.732 (best 0.732), test acc: 0.721 (best 0.721)
In epoch 35, loss: 1.104, val acc: 0.744 (best 0.744), test acc: 0.739 (best 0.734)
In epoch 40, loss: 0.930, val acc: 0.754 (best 0.754), test acc: 0.748 (best 0.748)
In epoch 45, loss: 0.767, val acc: 0.758 (best 0.758), test acc: 0.751 (best 0.752)
In epoch 50, loss: 0.624, val acc: 0.766 (best 0.766), test acc: 0.756 (best 0.756)
In epoch 55, loss: 0.502, val acc: 0.768 (best 0.768), test acc: 0.759 (best 0